In [2]:
import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords

In [3]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [4]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [5]:
#Vamos a pasar a 0 y 1 la variable objetivo

df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [6]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [7]:
df_raw['url'].sample(20)

1860    https://www.npr.org/sections/live-updates-prot...
57      https://www.supremecourt.gov/opinions/19pdf/18...
2062     https://bigspaceship.us9.list-manage.com/profile
2692    https://www.vanityfair.com/hollywood/2020/06/t...
1231    https://search.itunes.apple.com/WebObjects/MZC...
2959    https://melmagazine.com/en-us/story/who-is-o-m...
843     https://www.reuters.com/article/us-serbia-russ...
463      https://lovevery.com/pages/the-play-kits-program
101            https://www.morningbrew.com/emerging-tech/
1000     https://briefingday.com/n/20200625/m#commentform
1252    https://digg.com/2020/best-memes-trump-walk-of...
2354                                    https://fat.gold/
2899             https://wtf.us10.list-manage.com/profile
1687                               https://www.hvper.com/
1704    https://www.marketwatch.com/story/7-reasons-th...
205       https://en.wikipedia.org/wiki/Culinary_triangle
2493    https://www.youtube.com/watch?v=TrIqmz5MRQs&fe...
1538    https:

In [8]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [10]:
# Vamos a ver como esta nuestra variable objetivo en este caso es is_spam
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("No son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)

Hay spam:  696
No son spam :  2303
(2999, 2)


In [11]:
# Eliminar los duplicados

df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

In [12]:
df_raw['url'] = df_raw['url'].str.lower()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

In [ ]:
df_aux = df_raw.copy()

In [ ]:
# Proceso de limpieza de datos

limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #savar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#assign the cleaned descriptions to the data frame
df_aux['url'] = limpieza
        
df_aux.head()

In [ ]:

df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

In [ ]:

# Creamos una lista de stopwords
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [ ]:
def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result 

In [ ]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [ ]:
df_aux.sample(10)

In [ ]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

In [ ]:
# Ahora que hemos limpiado nuestros datos
# procedemos a hacer una copia con el dataset a trabajar en el final
df = df_aux.copy()

In [ ]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [ ]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [ ]:
print(X_train)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:

print(vec.get_feature_names_out())

In [ ]:
nb = MultinomialNB()

In [ ]:
nb.fit(X_train, y_train)

In [ ]:
nb.score(X_train,y_train)

In [ ]:

nb.score(X_test,y_test)

In [ ]:

predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
#Creando la Matriz

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.40, random_state = 121, shuffle = True)

In [ ]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [ ]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
pickle.dump(cl, open('../models/texto_NLP.pkl', 'wb'))

In [ ]:
# Si queremos cargar el archivo guardado en la carpeta models

load_model = pickle.load(open('../models/texto_NLP.pkl', 'rb'))

In [ ]:
load_model